In [1]:
from osgeo import gdal
import numpy as np

In [2]:
def crop_by_slide_window(image_path, output_folder, window_size, step_size, image_class):
    global S2_count
    global MODIS_count
    global S1_count
    global ref_count
    global before_count
    global after_count

    ds = gdal.Open(image_path)
    if ds is None:
        print(f"No such path: {image_path}")
        return False

    # 获取图像的宽度和高度
    width = ds.RasterXSize
    height = ds.RasterYSize

    # 计算滑动窗口的数量
    num_windows_width = (width - window_size) // step_size + 1
    num_windows_height = (height - window_size) // step_size + 1

    # 对每个滑动窗口进行裁剪
    for i in range(num_windows_width):
        for j in range(num_windows_height):
            if image_class == "S2":
                count = S2_count
                S2_count += 1
            elif image_class == "S1":
                count = S1_count
                S1_count += 1
            elif image_class == "MODIS":
                count = MODIS_count
                MODIS_count += 1
            elif image_class == "ref":
                count = ref_count
                ref_count += 1
            elif image_class == "before":
                count = before_count
                before_count += 1
            elif image_class == "after":
                count = after_count
                after_count += 1

            # 计算滑动窗口的起始坐标
            start_x = i * step_size
            start_y = j * step_size

            # 读取滑动窗口中的数据
            image = ds.ReadAsArray(start_x, start_y, window_size, window_size)

            # 创建新的tif文件来保存裁剪后的图像
            driver = gdal.GetDriverByName('GTiff')
            out_ds = driver.Create(output_folder + image_class + f"_{count}.tif", window_size, window_size, ds.RasterCount, gdal.GDT_Int16)

            # 将裁剪后的图像数据写入新的tif文件
            for k in range(ds.RasterCount):
                out_band = out_ds.GetRasterBand(k + 1)
                out_band.WriteArray(image[k])

            # 设置新tif文件的地理变换参数和投影信息
            geo_transform = list(ds.GetGeoTransform())
            geo_transform[0] += start_x * geo_transform[1]
            geo_transform[3] += start_y * geo_transform[5]
            out_ds.SetGeoTransform(geo_transform)
            out_ds.SetProjection(ds.GetProjection())

            # 关闭新tif文件
            out_ds = None

    # 关闭原始tif文件
    ds = None

    print("Successfully crop " + image_path)
    return True

In [3]:
import pandas as pd

file_path = r'D:\ENVI\data\NingBo\NB_roi\data_time_matching.xlsx'
excel_file = pd.ExcelFile(file_path)

sheet_names = excel_file.sheet_names

columns_to_read = ['S2_target', 'MODIS', 'S1', 'S2_reference', 'S2_interpolation_before', 'S2_interpolation_after']

image_date_dict = {}

for sheet_name in sheet_names:
    df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=columns_to_read)
    image_date_dict[sheet_name] = df
    

In [4]:
print(image_date_dict.keys())

dict_keys(['NB_roi_1', 'NB_roi_2', 'NB_roi_3', 'NB_roi_4', 'NB_roi_5', 'NB_roi_6', 'NB_roi_7', 'NB_roi_8', 'NB_roi_test'])


In [4]:
S2_count = 0
MODIS_count = 0
S1_count = 0
ref_count = 0
before_count = 0
after_count = 0

for roi in image_date_dict.keys():
    S2_date = image_date_dict[roi]["S2_target"].tolist()
    MODIS_l_date = image_date_dict[roi]["MODIS"].tolist()
    MODIS_h_date = MODIS_l_date
    S1_date = image_date_dict[roi]["S1"].tolist()
    S2_ref_date = image_date_dict[roi]["S2_reference"].tolist()
    S2_before_date = image_date_dict[roi]["S2_interpolation_before"].tolist()
    S2_after_date = image_date_dict[roi]["S2_interpolation_after"].tolist()

    S2_image_path = [r"D:\ENVI\data\NingBo\NB_roi\raw_data\S2\{}\{}_S2_{}.tif".format(roi, roi, date) for date in S2_date]
    MODIS_l_image_path = [r"D:\ENVI\data\NingBo\NB_roi\raw_data\MODIS_500\{}\{}_MODIS_{}.tif".format(roi, roi, date) for date in MODIS_l_date]
    MODIS_h_image_path = [r"D:\ENVI\data\NingBo\NB_roi\raw_data\MODIS_250\{}\{}_MODIS_{}.tif".format(roi, roi, date) for date in MODIS_h_date]
    S1_image_path = [r"D:\ENVI\data\NingBo\NB_roi\raw_data\S1\{}\{}_S1_{}.tif".format(roi, roi, date) for date in S1_date]
    ref_image_path = [r"D:\ENVI\data\NingBo\NB_roi\raw_data\S2\{}\{}_S2_{}.tif".format(roi, roi, date) for date in S2_ref_date]
    before_image_path = [r"D:\ENVI\data\NingBo\NB_roi\raw_data\S2\{}\{}_S2_{}.tif".format(roi, roi, date) for date in S2_before_date]
    after_image_path = [r"D:\ENVI\data\NingBo\NB_roi\raw_data\S2\{}\{}_S2_{}.tif".format(roi, roi, date) for date in S2_after_date]

    usage = "test" if roi == 'NB_roi_test' else "train"
    print(usage)

    for S2_path, MODIS_500_path, MODIS_250_path, S1_path, ref_path, before_path, after_path in zip(S2_image_path, MODIS_l_image_path, MODIS_h_image_path, S1_image_path, ref_image_path, before_image_path, after_image_path):
        # crop_by_slide_window(S2_path, r"D:\ENVI\data\NingBo\NB_roi\cropped_data\\" + usage + r"\S2\\", 250, 200, "S2")
        # crop_by_slide_window(MODIS_500_path, r"D:\ENVI\data\NingBo\NB_roi\cropped_data\\" + usage + r"\MODIS_l\\", 5, 4, "MODIS")
        crop_by_slide_window(MODIS_250_path, r"D:\ENVI\data\NingBo\NB_roi\cropped_data\\" + usage + r"\MODIS_h\\", 10, 8, "MODIS")
        # crop_by_slide_window(S1_path, r"D:\ENVI\data\NingBo\NB_roi\cropped_data\\" + usage + r"\S1\\", 250, 200, "S1")
        # crop_by_slide_window(ref_path, r"D:\ENVI\data\NingBo\NB_roi\cropped_data\\" + usage + r"\ref\\", 250, 200, "ref")
        # crop_by_slide_window(before_path, r"D:\ENVI\data\NingBo\NB_roi\cropped_data\\" + usage + r"\before\\", 250, 200, "before")
        # crop_by_slide_window(after_path, r"D:\ENVI\data\NingBo\NB_roi\cropped_data\\" + usage + r"\after\\", 250, 200, "after")
        print(S2_count)
        print(MODIS_count)
        print(S1_count)
        print(ref_count)
        print(before_count)
        print(after_count)

    # print(roi)
    # print(S2_image_path, MODIS_image_path, S1_image_path)


train
Successfully crop D:\ENVI\data\NingBo\NB_roi\raw_data\MODIS_250\NB_roi_1\NB_roi_1_MODIS_20190509.tif
0
12
0
0
0
0
Successfully crop D:\ENVI\data\NingBo\NB_roi\raw_data\MODIS_250\NB_roi_1\NB_roi_1_MODIS_20190618.tif
0
24
0
0
0
0
Successfully crop D:\ENVI\data\NingBo\NB_roi\raw_data\MODIS_250\NB_roi_1\NB_roi_1_MODIS_20190728.tif
0
36
0
0
0
0
Successfully crop D:\ENVI\data\NingBo\NB_roi\raw_data\MODIS_250\NB_roi_1\NB_roi_1_MODIS_20190930.tif
0
48
0
0
0
0
Successfully crop D:\ENVI\data\NingBo\NB_roi\raw_data\MODIS_250\NB_roi_1\NB_roi_1_MODIS_20191101.tif
0
60
0
0
0
0
Successfully crop D:\ENVI\data\NingBo\NB_roi\raw_data\MODIS_250\NB_roi_1\NB_roi_1_MODIS_20191109.tif
0
72
0
0
0
0
Successfully crop D:\ENVI\data\NingBo\NB_roi\raw_data\MODIS_250\NB_roi_1\NB_roi_1_MODIS_20191117.tif
0
84
0
0
0
0
Successfully crop D:\ENVI\data\NingBo\NB_roi\raw_data\MODIS_250\NB_roi_1\NB_roi_1_MODIS_20191117.tif
0
96
0
0
0
0
Successfully crop D:\ENVI\data\NingBo\NB_roi\raw_data\MODIS_250\NB_roi_1\NB_roi_1_